In [1]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
import yfinance as yf
import polars as pl
from data_loading import preprocess_history
from model import Model
from huggingface_hub import HfApi
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

In [2]:
history = pl.from_pandas(yf.Ticker('GOOG').history(period="ytd"), include_index=True)

In [3]:
data = history[-32-8:]

In [4]:
data

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
"datetime[ns, America/New_York]",f64,f64,f64,f64,i64,f64,f64
2024-10-18 00:00:00 EDT,164.683157,166.181457,164.563298,164.862961,13091300,0.0,0.0
2024-10-21 00:00:00 EDT,164.393503,166.031644,164.118805,165.612122,11384000,0.0,0.0
2024-10-22 00:00:00 EDT,164.513358,167.280224,164.483393,166.630966,11958600,0.0,0.0
2024-10-23 00:00:00 EDT,166.241397,167.410084,163.447569,164.29361,12754300,0.0,0.0
2024-10-24 00:00:00 EDT,164.403481,164.862967,162.585552,164.343552,12764400,0.0,0.0
…,…,…,…,…,…,…,…
2024-12-09 00:00:00 EST,175.714996,178.039993,175.399994,177.100006,19887800,0.2,0.0
2024-12-10 00:00:00 EST,184.535004,188.029999,182.669998,186.529999,34317400,0.0,0.0
2024-12-11 00:00:00 EST,186.699997,196.889999,186.259995,196.710007,41664500,0.0,0.0


In [5]:
date_diffs = data.select((pl.col('Date') - pl.first('Date')).dt.total_days())

In [6]:
model = LinearRegression().fit(date_diffs[:-8].to_numpy(), data[:-8, 'Close'].to_numpy())

In [11]:
api = HfApi()
nn_model = Model.load_from_hf(api, 'itransformer-6')
nn_model.eval()

In [12]:
extend = 3
nn_data = preprocess_history(history.select(pl.all().name.to_lowercase())).to_jax()
forecast = nn_model.forecast(nn_data[-32-8+extend:-8+extend])[extend:]
pred_data = data.with_columns(pred=np.pad(np.array(forecast), (32 + extend, 0), constant_values=np.nan))

In [13]:
fig = px.line(pred_data, x='Date', y=['Close', 'pred'])
fig.add_vline(data[-8+extend, 'Date'])
fig